<a href="https://colab.research.google.com/github/wonjunchoi-arc/tensorflow/blob/main/%EA%B0%80%EC%9C%84%EB%B0%94%EC%9C%84%EB%B3%B4%2C_%ED%97%88%EB%A6%AC%EB%94%94%EC%8A%A4%ED%81%AC%2C%EB%AA%A9%EB%94%94%EC%8A%A4%ED%81%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab.patches import cv2_imshow
from cv2 import ml_KNearest
import cv2 
import mediapipe as mp
import numpy as np


max_num_hands = 1
gesture = {
0:'fist',1:'one',2:'two',3:'three',4:'four',5:'five',6:'six',
            7:'rock', 8:'spiderman', 9:'yeah', 10:'ok',

}

rps_gesture = {0: 'rock', 5:'paper', 9:'scissors'}

#Media
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands =max_num_hands,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5,
)

from sklearn.neighbors import KNeighborsClassifier
#Gesture R model
file =np.genfromtxt('../down/gesture_train.csv',delimiter=',')
angle =file[:,:-1].astype(np.float32)
label =file[:,-1].astype(np.float32)
print(angle)
print(label)
knn = ml_KNearest.create()
# knn = KNeighborsClassifier(n_neighbors=2)
# knn.fit(angle,label)/
knn.train(angle, cv2.ml.ROW_SAMPLE,label)


cap = cv2.imread('../down/0.png')

if cap is not None:
    cap= cv2.flip(cap, 1)
    cap = cv2.cvtColor(cap,cv2.COLOR_BGR2RGB)

    results =hands.process(cap)

    cap =cv2.cvtColor(cap, cv2.COLOR_RGB2BGR)
    cv2_imshow(cap)
else:
    print('No image file.')
print(len(results.multi_hand_landmarks))

if results.multi_hand_landmarks is not None:
    for res in results.multi_hand_landmarks:
        joint =np.zeros((21,3))
        for j, lm in enumerate(res.landmark):
            joint[j] = [lm.x, lm.y, lm.z]
v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:]
v2 =joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:]
v = v2 - v1
v= v/ np.linalg.norm(v, axis=1)[:,np.newaxis]


angle =np.arccos(np.einsum('nt,nt->n',
v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:],
v[[1,2,3,5,6,7,9,10,11,13,14,15,7,18,19],:]))


angle =np.degrees(angle)



data =np.array([angle], dtype=np.float32)
print(data)
ret, results, neighbors, dist = knn.findNearest(data, 3)
idx =int(results[0][0])
print(ret)
print(results)
print(neighbors)
print(dist)



#Draw
if idx in rps_gesture.keys():
    cv2.putText(cap, text=rps_gesture[idx].upper(),
                org=(int(res.landmark[0].x * cap.shape[1])
    , int(res.landmark[0].y * cap.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    fontScale=1, color=(255,255,255), thickness=2)

print('여기까지 실행')

mp_drawing.draw_landmarks(cap, res, mp_hands.HAND_CONNECTIONS)
cv2_imshow(cap)





In [ ]:
!pip  install mediapipe
from google.colab.patches import cv2_imshow
from cv2 import ml_KNearest
import cv2 
import mediapipe as mp
import numpy as np
import csv
import pandas as pd


max_num_human = 1
discs_pose = {
0:'Good_Pose',1:'NO!',2:'neck_discs',

}


#Media
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
     model_complexity=2,
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5,
)

#Gesture R model
file =np.genfromtxt('../down/discs_train.csv',delimiter=',')
angle =file[:,:-1].astype(np.float32)
label =file[:,-1].astype(np.float32)

knn = ml_KNearest.create()
knn.train(angle, cv2.ml.ROW_SAMPLE,label)

cap = cv2.imread('../down/nor/5.jpg')

if cap is not None:
    cap= cv2.flip(cap, 1)
    cap = cv2.cvtColor(cap,cv2.COLOR_BGR2RGB)

    results =pose.process(cap)

    cap =cv2.cvtColor(cap, cv2.COLOR_RGB2BGR)
    cv2_imshow(cap)
else:
    print('No image file.')

a=[]
a.append(results.pose_landmarks)
print(len(a))

if results.pose_landmarks is not None:
     for res in a:
         joint =np.zeros((33,4))
         for j, lm in enumerate(res.landmark):
            joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

v1 = joint[[11,11,12,12,11,12,23],:]
v2 =joint[[0,7,0,7,23,24,24],:]
v = v2 - v1
v= v/ np.linalg.norm(v, axis=1)[:,np.newaxis]


angle =np.arccos(np.einsum('nt,nt->n',
v[[0,1,2,3,4,5],:],
v[[1,2,3,4,5,6],:]))

angle =np.degrees(angle)

# print(type(angle))

# df = pd.DataFrame(angle)
# df.loc[6] = [1]
# df=df.transpose()
# print(df)

# df.to_csv('../down/discs_train.csv',mode='a', index=False,header = False)

data =np.array([angle], dtype=np.float32)
ret, results, neighbors, dist = knn.findNearest(data, 1)
idx =int(results[0][0])



#Draw
if idx in discs_pose.keys():
    cv2.putText(cap, text=discs_pose[idx].upper(),
                org=(int(res.landmark[25].x * cap.shape[1])
    , int(res.landmark[0].y * cap.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    fontScale=0.5, color=(255,100,120), thickness=1)


mp_drawing.draw_landmarks(cap, res, mp_pose.POSE_CONNECTIONS)
cv2_imshow(cap)

